# Importing Libraries

In [1]:
import pickle
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
from sklearn.preprocessing import RobustScaler

# Importing Data

In [2]:
with open('new_eda_data.pkl', 'rb') as file:
    df = pickle.load(file)

In [3]:
# select columns for numerical model
df = df[['licensed_content', 'subscribers', 'total_views', 'total_videos', 'category', 'no_of_tags', 'title_length',
       'description_length', 'target', 'age', 'duration_minutes']]

# Train Test Split

In [4]:
# features
X = df.drop(columns=['target'])
# target
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

# Feature Engineering

In [5]:
# create function to one hot encode the train and test features respectively
def ohe(train, test):

    # map licensed content to binary for train
    train['licensed_map'] = train.licensed_content.apply(lambda x: 1 if x == True else 0)

    # map licensed content to binary for test
    test['licensed_map'] = test.licensed_content.apply(lambda x: 1 if x == True else 0)

    # store unique categories
    unique_categories = set()

    # iterate through unique categories
    for category in train['category']:
        unique_categories.add(category)

    # sort unique categories alphabetically
    unique_categories = sorted(unique_categories)

    # create a binary mapping of all unique categories in train
    for column in unique_categories:
        train[column] = train['category'].apply(lambda x: 1 if column == x else 0)

    # create a binary mapping of all unique categories in test
    for column in unique_categories:
        test[column] = test['category'].apply(lambda x: 1 if column == x else 0)
    
    # drop columns
    train.drop(columns=['category', 'licensed_content'], inplace=True)
    test.drop(columns=['category', 'licensed_content'], inplace=True)

    return train, test

In [6]:
def rob_scaler(train, test, target_train, target_test, columns_to_scale, other_columns):

    # train
    train_scale = train[columns_to_scale] ## columns to scale
    train_not_scale = train[other_columns] ## remaining columns

    rob = RobustScaler() ## scaler object
    rob.fit(train_scale) ## fit the scaler with train data

    train_rob = pd.DataFrame(rob.transform(train_scale), columns=train_scale.columns) ## create df with transformed training data

    train_not_scale.reset_index(drop=True, inplace=True) ## Reset X_train index
    target_train.reset_index(drop=True, inplace=True) ## Reset y_train index

    train_fe_rob = pd.concat([train_not_scale, train_rob], axis=1) ## concatenate scaled data with remaining columns

    # test
    test_scale = test[columns_to_scale] ## columns to scale
    test_not_scale = test[other_columns] ## remaining columns

    test_rob = pd.DataFrame(rob.transform(test_scale), columns=test_scale.columns) ## create df with transformed test data

    test_not_scale.reset_index(drop=True, inplace=True) ## Reset X_test index
    target_test.reset_index(drop=True, inplace=True) ## Reset y_test index

    test_fe_rob = pd.concat([test_not_scale, test_rob], axis=1) ## concatenate scaled data with remaining columns

    return train_fe_rob, test_fe_rob, target_train, target_test

In [7]:
# list of numerical columns to scale
scale_columns = ['subscribers', 'total_views', 'total_videos', 'no_of_tags', 'title_length', 'description_length', 'age', 'duration_minutes']

# list of remaining columns
non_scale_columns = ['licensed_map', 'Education', 'Entertainment', 'Film & Animation', 'Gaming','Howto & Style',
                      'Music', 'People & Blogs', 'Science & Technology', 'Sports', 'Travel & Events']

In [8]:
# one hot encode / map selected categorical features
X_train_fe, X_test_fe = ohe(X_train, X_test)

In [9]:
# scale selected continuous features
X_train_fe_rob, X_test_fe_rob, y_train_fe, y_test_fe = rob_scaler(X_train_fe, X_test_fe, y_train, y_test, scale_columns, non_scale_columns)

# Modelling

### Default RF

In [25]:
rf = RandomForestClassifier(random_state=21)
rf.fit(X_train_fe_rob, y_train_fe)

RandomForestClassifier(random_state=21)

In [26]:
y_train_pred = rf.predict(X_train_fe_rob)
y_test_pred = rf.predict(X_test_fe_rob)

In [27]:
# function to calculate the accuracy, precision and recall
def apr(y_pred, y_real):       
    
    """ Calculates accuracy, precision, recall
        Requires predicted value first, and then the real value
    """
    accuracy = metrics.accuracy_score(y_real, y_pred)
    precision = metrics.precision_score(y_real, y_pred)
    recall = metrics.recall_score(y_real, y_pred)
    f1 = metrics.f1_score(y_real, y_pred)

    print(f"Accuracy:{accuracy}")
    print(f"Precision:{precision}")
    print(f"Recall:{recall}")
    print(f"F1:{f1}")

In [28]:
# train metrics
apr(y_train_pred, y_train)

Accuracy:1.0
Precision:1.0
Recall:1.0
F1:1.0


In [29]:
# test metrics
apr(y_test_pred, y_test)

Accuracy:0.8365667254556144
Precision:0.8354725787631272
Recall:0.839390386869871
F1:0.8374269005847953


### Tuned RF

In [41]:
rf = RandomForestClassifier(random_state=21)

In [52]:
# inputting the parameter values
rf_params = {        
    'max_depth': [7],
    'n_estimators': [220, 225, 230, 235, 240],
    'min_samples_split': [6],
    'min_samples_leaf': [5]
}

# finding the optimal parameter values for the model
gs = GridSearchCV(rf, param_grid=rf_params, cv=5, scoring = 'accuracy') # choosing accuracy as the best metric because our train data is class balanced

# fitting the train data into the model                                                                                        
gs.fit(X_train_fe_rob, y_train)                                           
gs.best_params_

{'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 6,
 'n_estimators': 235}

In [43]:
y_pred_train = gs.predict(X_train_fe_rob)
y_pred_test = gs.predict(X_test_fe_rob)

In [44]:
# train metrics
apr(y_pred_train, y_train)

Accuracy:0.8300749669263561
Precision:0.8313110387688666
Recall:0.8273932253313696
F1:0.8293475051668143


In [51]:
# test metrics
apr(y_pred_test, y_test)

Accuracy:0.818342151675485
Precision:0.8126436781609195
Recall:0.8288393903868698
F1:0.820661636680209


# Pickling

In [48]:
# final model
gs.best_estimator_

RandomForestClassifier(max_depth=7, min_samples_leaf=5, min_samples_split=6,
                       n_estimators=235, random_state=21)

In [49]:
with open ('numerical_rf.pkl', 'wb') as file:
    pickle.dump(gs.best_estimator_, file)